# Анализ результатов

In [1]:
import pandas as pd

results_df = pd.read_csv("../data/test/final/result_df.csv")

In [2]:
results_df.tail()

,Unnamed: 0,question,is_fc,city_from,city_to,date,date_back,need_luggage,answer,is_fc_answer,parsed_fc,function_name,parsed_args,parsed_city_from,parsed_city_to,parsed_date,parsed_date_back,parsed_need_luggage,all_parsed_arguments
226,226,How do pilots navigate through turbulent weat...,False,NaN,NaN,NaN,NaN,NaN,PILOTS NAVIGATE THROUGH TURBULENT WEATHER CON...,False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
227,227,How have advancements in technology impacted ...,False,NaN,NaN,NaN,NaN,NaN,Advancements in technological developments on...,False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
228,228,What is the average altitude at which commerci...,False,NaN,NaN,NaN,NaN,NaN,"I'm sorry, but I can only provide information ...",False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
229,229,How do pilots navigate through turbulence dur...,False,NaN,NaN,NaN,NaN,NaN,"As an AI, I don't have access to external dat...",False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
230,230,What is the role of pilots in ensuring safe fl...,False,NaN,NaN,NaN,NaN,NaN,"Piloting, or flying as it appears to be common...",False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
results_df.head()

,Unnamed: 0,question,is_fc,city_from,city_to,date,date_back,need_luggage,answer,is_fc_answer,parsed_fc,function_name,parsed_args,parsed_city_from,parsed_city_to,parsed_date,parsed_date_back,parsed_need_luggage,all_parsed_arguments
0,0,I am writing to request the purchase of a roun...,True,Moscow,Novosibirsk,09.20.2022,12.15.2022,True,"{""name"": ""get_flight_info"", ""description"": ""Ge...",True,True,get_flight_info,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,I would like to purchase a round-trip airfare ...,True,Moscow,Novosibirsk,09.20.2022,12.15.2022,True,"{""name"": ""get_flight_info"", ""arguments"": '{""_f...",True,True,get_flight_info,True,NaN,NaN,"{'departure': 20, 'arrival': 31}",NaN,NaN,"['_from', 'to', 'date']"
2,2,I am writing to request the purchase of a roun...,True,Moscow,Novosibirsk,09.20.2022,12.15.2022,NaN,I'm sorry if my previous response caused any c...,False,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
3,3,I would like to purchase a round-trip air tick...,True,Moscow,Novosibirsk,09.20.2022,12.15.2022,NaN,"{""name"": ""get_flight_info"", ""arguments"": '{\n""...",True,True,get_flight_info,True,Moscow,Novosibirsk,20-Sep-20,NaN,NaN,"['city_from', 'city_to', 'date']"
4,4,I would like to purchase a round trip flight t...,True,Moscow,Novosibirsk,09.20.2022,NaN,True,"{""name"": ""get_flight_info"", ""arguments"": '{""ci...",True,True,get_flight_info,True,Moscow,Novosibirsk,09-15-2022,NaN,NaN,"['city_from', 'city_to', 'date']"


## Function call prediction

In [4]:
# accuracy обнаружение function call
sum(results_df["is_fc"] == results_df["is_fc_answer"]) / len(results_df)

0.9393939393939394

In [5]:
# recall обнаружение function call
real_fc = results_df["is_fc"] == True
fc_total = sum(real_fc)
print(fc_total)

# выбираем из реально fc те, что модель предсказала как fc
sum(results_df[real_fc]["is_fc_answer"] == True) / fc_total

118


0.9067796610169492

In [6]:
# precision обнаружение function call
pred_fc = results_df["is_fc_answer"] == True
fc_preds_total = sum(pred_fc)
print(fc_preds_total)

# # выбираем из тех, что модель предсказала как fc, реальные fc
sum(results_df[pred_fc]["is_fc"] == True) / fc_preds_total

110


0.9727272727272728

## Parsing json

In [7]:
# сколько получилось распарсить json из тех, что определили как fc
p = sum(results_df[pred_fc]["parsed_fc"])
print(p, p / fc_preds_total)

100 0.9090909090909091


In [8]:
# сколько получилось распарсить json  и аргументы json из тех, что определили как fc
p = sum(results_df[pred_fc]["parsed_args"])
print(p, p / fc_preds_total)

96 0.8727272727272727


## Identifying arguments

Теперь посмотрим на самое главное - на то, как модель справляется с определением аргументов

Выберем true function calls - и по каждому из аргументов сравним, как получилось достать из данных нужную информацию



In [9]:
results_df[real_fc]["city_from"]

0             Moscow
1             Moscow
2             Moscow
3             Moscow
4             Moscow
           ...      
113    Yekaterinburg
114    Yekaterinburg
115    Yekaterinburg
116    Yekaterinburg
117    Yekaterinburg
Name: city_from, Length: 118, dtype: object

In [10]:
# accuracy идентицикации city_from
sum(
    results_df[real_fc]["city_from"] == results_df[real_fc]["parsed_city_from"]
) / fc_preds_total

0.7181818181818181

In [11]:
# accuracy идентицикации city_to
sum(
    results_df[real_fc]["city_to"] == results_df[real_fc]["parsed_city_to"]
) / fc_preds_total

0.45454545454545453

In [12]:
# accuracy идентицикации need_luggage
sum(
    results_df[real_fc]["need_luggage"] == results_df[real_fc]["parsed_need_luggage"]
) / fc_preds_total

0.05454545454545454

In [13]:
sum(~results_df[real_fc]["parsed_need_luggage"].isna()), sum(
    ~results_df[real_fc]["need_luggage"].isna()
)

(7, 38)

С датами немного поработаем - они изначально в разных форматах

В тестовых данных они в формате %m.%d.%Y, а модель должна была выдавать (формат указан в system prompt) в формате MM-DD-YYYY

In [14]:
from datetime import datetime


def format_model_date(text: str | None):
    if text is None:
        return None
    for format in ("%m-%d-%Y", "%m/%d/%Y", "%m.%d.%Y"):
        try:
            return datetime.strptime(text, format)
        except ValueError:
            pass
    return None


def format_test_date(text: str | None):
    if text is None:
        return None
    return datetime.strptime(text, "%m.%d.%Y")

### Arrival date

In [15]:
real_dates = results_df[real_fc]["date"].apply(
    lambda x: format_test_date(str(x)) if not pd.isna(x) else None
)
pred_dates = results_df[real_fc]["parsed_date"].apply(
    lambda x: format_model_date(str(x)) if not pd.isna(x) else None
)

# Дат в оригинале
print(sum(~real_dates.isna()))
# Всего изначально модели удалось спарсить даты
print(sum(~results_df[real_fc]["parsed_date"].isna()))
# И сколько получилось распарсить строги форматом "%m-%d-%Y"
print(sum(~pred_dates.isna()))
# Из них совпали
print(sum(pred_dates == real_dates))

96
76
39
21


In [16]:
results_df[real_fc]["parsed_date"].value_counts()

parsed_date
11-28-2021                          8
12-04-2020                          6
09-15-2021                          4
11-17-2022                          3
09-20-2022                          3
09/15/2021                          2
09-20                               2
17-11-2022                          2
12-30                               2
12/04/2020                          2
2021-11-28                          2
09-15-2022                          2
20-Sep-20                           1
{'departure': 20, 'arrival': 31}    1
09-20-22                            1
20-09-2022                          1
09-17-2022                          1
20-Sep-2022                         1
11-15-22                            1
11/07/2023                          1
11/0723                             1
11/20                               1
11-23                               1
11/06                               1
10-25-2023                          1
10-09-20                            1


### Departure date

In [17]:
real_dates = results_df[real_fc]["date_back"].apply(
    lambda x: format_test_date(str(x)) if not pd.isna(x) else None
)
pred_dates = results_df[real_fc]["parsed_date_back"].apply(
    lambda x: format_model_date(str(x)) if not pd.isna(x) else None
)
# Дат в оригинале
print(sum(~real_dates.isna()))
# Всего изначально модели удалось спарсить даты
print(sum(~results_df[real_fc]["parsed_date_back"].isna()))
# И сколько получилось распарсить строги форматом "%m-%d-%Y"
print(sum(~pred_dates.isna()))
# Из них совпали
print(sum(pred_dates == real_dates))

38
4
3
3


In [18]:
results_df[real_fc][~results_df[real_fc]["parsed_date_back"].isna()]

,Unnamed: 0,question,is_fc,city_from,city_to,date,date_back,need_luggage,answer,is_fc_answer,parsed_fc,function_name,parsed_args,parsed_city_from,parsed_city_to,parsed_date,parsed_date_back,parsed_need_luggage,all_parsed_arguments
12,12,I would like to purchase round-trip plane tick...,True,Moscow,Novosibirsk,09.20.2022,12.15.2022,False,"{""name"": ""get_flight_info"", ""arguments"": '{\n ...",True,True,get_flight_info,True,Moscow,Novosibirsk,09-20-2022,12-15-2022,False,"['city_from', 'city_to', 'date', 'date_back', ..."
50,50,I would like to purchase a round trip airfare ...,True,Novosibirsk,Moscow,11.28.2021,03.01.2023,True,"{""name"": ""get_flight_info"", ""arguments"": '{""ci...",True,True,get_flight_info,True,Novosibirsk,Moscow,11-28-2021,“3-5”,True,"['city_from', 'city_to', 'date', 'date_back', ..."
63,63,I would like to purchase a flight ticket from ...,True,Novosibirsk,Moscow,11.28.2021,03.01.2023,False,"{""name"": ""get_flight_info"", ""arguments"": '{""ci...",True,True,get_flight_info,True,Novosibirsk,Moscow,11-28-2021,3-1-2023,False,"['city_from', 'city_to', 'date', 'date_back', ..."
83,83,I would like to purchase a flight ticket from ...,True,Yekaterinburg,Saint-Petersburg,09.15.2021,11.05.2022,False,"{""name"": ""get_flight_info"", ""arguments"": '{\n ...",True,True,get_flight_info,True,Yekaterinburg,Saint-Petersburg,09-15-2021,11-05-2022,NaN,"['city_from', 'city_to', 'date', 'date_back']"
